In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-13"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10312,2024-05-13,Rússia Liga Vtb United,13:30,Lokomotiv Kuban,Unics Kazan,1.58,2.40,163.5,1.82,1.94,0.0,0.00,0.00,O84uSLYI,0.632911,0.416667,0.549451,0.515464,0.049578,167.798,56.133794,0.334532,2.4,1.341641,0.559017,151.30,2.008,0.780750,0.388820,81.0,120.072,36.522163,0.304169,1.8,1.643168,0.912871,180.11,1.498,0.401709,0.268163,20.0,89,83,1.70,2.17,130.232,159.448,0.291371,0.045134,NaN,-0.87,-0.174,-3.333333,0.728887,0.7,-0.028887,-2.28,-0.456,-3.070175,0.613432,0.5,-0.113432
10313,2024-05-13,Paraguai Lnb,21:30,Colonias Gold,Sol De America,1.04,22.00,143.5,1.88,1.82,0.0,0.00,0.00,pjH3Bxse,0.961538,0.045455,0.531915,0.549451,0.006993,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,58,52,3.02,7.28,0.000,0.000,1.286542,0.022933,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10314,2024-05-13,Portugal Lpb,15:00,Imortal,FC Porto,6.69,1.08,164.5,1.81,1.85,0.0,0.00,0.00,WfmYqNdK,0.149477,0.925926,0.552486,0.540541,0.075403,353.812,314.308464,0.888349,0.0,0.000000,NaN,887.50,4.974,4.552618,0.915283,-78.0,100.136,7.766420,0.077559,2.4,1.341641,0.559017,91.80,1.120,0.131149,0.117097,66.0,71,90,12.50,1.02,299.244,107.056,1.021073,0.015456,NaN,-2.41,-0.482,-11.804979,0.000000,0.0,0.000000,-0.64,-0.128,-0.625000,0.787359,0.6,-0.187359
10315,2024-05-13,Portugal Lpb,17:00,Ovarense,Sporting CP,2.20,1.62,168.5,1.86,1.84,0.0,0.00,0.00,IFmniv4m,0.454545,0.617284,0.537634,0.543478,0.071829,220.370,140.882628,0.639300,2.4,1.341641,0.559017,392.37,2.494,1.622338,0.650496,35.0,149.876,73.174655,0.488235,1.2,1.643168,1.369306,95.58,1.956,1.166653,0.596449,1.0,87,81,4.51,1.18,226.412,149.514,0.214724,0.007644,NaN,2.33,0.466,2.575107,0.000000,0.0,0.000000,-1.79,-0.358,-1.731844,0.000000,0.0,0.000000
10316,2024-05-13,Romênia Divisão A,13:00,CSM Targu Jiu,Laguna Bucuresti,1.07,7.23,158.5,1.83,1.85,0.0,0.00,0.00,pnc6Pr0k,0.934579,0.138313,0.546448,0.540541,0.072892,133.342,39.425644,0.295673,0.0,0.000000,NaN,78.75,1.616,0.459598,0.284404,-50.0,517.702,199.257403,0.384888,0.6,1.341641,2.236068,674.70,7.200,2.120094,0.294458,-59.0,75,78,1.05,8.65,106.450,633.518,1.049585,0.007686,NaN,-2.53,-0.506,-0.138340,0.656122,0.4,-0.256122,3.65,0.730,8.534247,0.109689,0.1,-0.009689
10317,2024-05-13,Romênia Divisão A,14:00,Petrolul Ploiesti,CSM Focsani,1.20,4.27,158.5,1.84,1.86,0.0,0.00,0.00,EoNn8SCO,0.833333,0.234192,0.543478,0.537634,0.067525,120.738,21.895598,0.181348,2.4,1.341641,0.559017,125.73,1.370,0.297658,0.217268,70.0,268.892,119.758078,0.445376,1.8,1.643168,0.912871,215.33,3.096,1.142248,0.368943,-14.0,99,61,1.27,3.53,118.134,375.734,0.793718,0.007644,NaN,0.10,0.020,10.000000,0.703919,0.6,-0.103919,4.54,0.908,3.601322,0.333201,0.4,0.066799
10318,2024-05-13,Romênia Divisão A,13:00,Voluntari,Arges Pitesti,1.31,3.25,158.5,1.87,1.79,0.0,0.00,0.00,fF5aG9KG,0.763359,0.307692,0.534759,0.558659,0.071051,188.616,63.729739,0.337881,1.8,1.643168,0.912871,110.88,2.362,0.930924,0.394125,-2.0,420.164,268.386936,0.638767,0.6,1.341641,2.236068,214.40,5.818,3.720849,0.639541,-62.0,84,67,1.32,3.20,141.628,413.134,0.601661,0.030912,NaN,1.50,0.300,1.033333,0.678907,0.7,0.021093,0.78,0.156,14.423077,0.000000,0.0,0.000000
10319,2024

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
10312,13:30,Rússia Liga Vtb United,Lokomotiv Kuban,Unics Kazan,1.58,Back Home
10331,21:00,Porto Rico Bsn,Vaqueros de Bayamon,Mets de Guaynabo,1.62,Back Home
